<a href="https://colab.research.google.com/github/Josh-XS/sales-predictions/blob/main/sales_predictions_machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project1

Joshua Nickell

##Loading Data

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
set_config(display='diagram')

In [ ]:
#load the data
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


##Data Cleaning

###Identify Duplicates

In [ ]:
df.duplicated().sum()

0

There are no duplicates.

###Drop Unnescesary Columns

In [ ]:
df.drop(columns = ['Item_Identifier','Outlet_Identifier','Outlet_Establishment_Year'], axis = 1, inplace = True)
df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,Medium,Tier 1,Supermarket Type1,3735.1380
1,5.92,Regular,0.019278,Soft Drinks,48.2692,Medium,Tier 3,Supermarket Type2,443.4228
2,17.50,Low Fat,0.016760,Meat,141.6180,Medium,Tier 1,Supermarket Type1,2097.2700
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,NaN,Tier 3,Grocery Store,732.3800
4,8.93,Low Fat,0.000000,Household,53.8614,High,Tier 3,Supermarket Type1,994.7052


These 3 columns would have no impact on our target, which is why they were dropped.

### Check For Inconsistencies

I need to create a loop to find any inconsistencies in the columns.

In [ ]:
cols = df.dtypes.index
for col in cols:
    print(f'- {col}:')
    print(df[col].value_counts(dropna=False))
    print("\n\n")

- Item_Weight:
NaN       1463
12.150      86
17.600      82
13.650      77
11.800      76
          ... 
7.275        2
7.685        1
9.420        1
6.520        1
5.400        1
Name: Item_Weight, Length: 416, dtype: int64



- Item_Fat_Content:
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64



- Item_Visibility:
0.000000    526
0.076975      3
0.162462      2
0.076841      2
0.073562      2
           ... 
0.013957      1
0.110460      1
0.124646      1
0.054142      1
0.044878      1
Name: Item_Visibility, Length: 7880, dtype: int64



- Item_Type:
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others     

### Fix Inconsistencies

In [ ]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('LF', 'Low Fat')

In [ ]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('low fat', 'Low Fat')

In [ ]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace('reg', 'Regular')

In [ ]:
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

###Ordinal Encoding

One of our categorical columns is ordinal, so I have decided to change it before splitting the data.

In [ ]:
df['Outlet_Size'].replace({'Small':0,'Medium':1,'High':2}, inplace=True)
df['Outlet_Size'].value_counts()

1.0    2793
0.0    2388
2.0     932
Name: Outlet_Size, dtype: int64

## Pre-Processing

###Splitting the Data

In [ ]:
#split data
X = df.drop(columns = 'Item_Outlet_Sales')
y = df['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
len(X_train)

6392

In [ ]:
len(X_test)

2131

In [ ]:
len(y_train)

6392

In [ ]:
len(y_test)

2131

## Creating Pipelines

In [ ]:
#create categorical and numerical selectors
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [ ]:
#create categorical and numerical imputers
mf_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer= SimpleImputer(strategy='mean')
#create scaler for numerical values
scaler = StandardScaler()
#create onehotencoder for nominal values
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [ ]:
#create numerical pipeline
num_pipeline = make_pipeline(mean_imputer, scaler)
num_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [ ]:
#create categorical pipeline
cat_pipeline = make_pipeline(mf_imputer, ohe)
cat_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

##Creating the Column Transformer

In [ ]:
#create tuples for numeric and categorical processors
nums = (num_pipeline, num_selector)
cats = (cat_pipeline, cat_selector)
#create column transformer with tuples
column_transformer = make_column_transformer(nums, cats, remainder = 'passthrough')

In [ ]:
#fit the column transformer on training data only
column_transformer.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f834631f710>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f834631f3d0>)])

In [ ]:
#transform training and testing data
X_train = column_transformer.transform(X_train)
X_test = column_transformer.transform(X_test)
#view numpy array of data
X_train

array([[ 0.81724868, -0.71277507,  1.82810922, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.5563395 , -1.29105225,  0.60336888, ...,  0.        ,
         1.        ,  0.        ],
       [-0.13151196,  1.81331864,  0.24454056, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.11373638, -0.92052713,  1.52302674, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.76600931, -0.2277552 , -0.38377708, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.81724868, -0.95867683, -0.73836105, ...,  1.        ,
         0.        ,  0.        ]])

##Creating the Model

In [ ]:
#create linear regression model
reg_model = LinearRegression()
#fit training data on model
reg_model.fit(X_train, y_train)

LinearRegression()

##Make Predictions

In [ ]:
#use model to predict 
train_p = reg_model.predict(X_train)
test_p = reg_model.predict(X_test )

In [ ]:
#print predictions
print(f'Training Predictions\n{train_p}\n')
print(f'Testing Predictions\n{test_p}\n')

Training Predictions
[3809. 2657. 2564. ... 3848. 1905. 1596.]

Testing Predictions
[1349.   776.5  834.5 ... 3414.  1976.5 2433. ]



##Creating a Function

I am going to create a functinon that gives us all of the accuracy metrics for a given model.

In [ ]:
def get_metrics(model):
  #find R2
  train_r2 = r2_score(y_train, train_p)
  test_r2 = r2_score(y_test, test_p)
  #print R2
  print(f'Training R2 - {train_r2}')
  print(f'Testing R2 - {test_r2}\n')

  #find MAE
  train_MAE = mean_absolute_error(y_train, train_p)
  test_MAE = mean_absolute_error(y_test, test_p)
  #print MAE
  print(f'Training MAE - {train_MAE}')
  print(f'Testing MAE - {test_MAE}\n')

  #find MSE
  train_MSE = mean_squared_error(y_train, train_p)
  test_MSE = mean_squared_error(y_test, test_p)
  #print MSE
  print(f'Training MSE - {train_MSE}')
  print(f'Testing MSE - {test_MSE}\n')

  #find RMSE
  train_RMSE = np.sqrt(train_MSE)
  test_RMSE = np.sqrt(test_MSE)
  #print RMSE
  print(f'Training RMSE - {train_RMSE}')
  print(f'Testing RMSE - {test_RMSE}\n')

In [ ]:
#apply function to model
get_metrics(reg_model)

Training R2 - 0.5605682085766622
Testing R2 - 0.5657150148289058

Training MAE - 847.5511586357948
Testing MAE - 805.6106213045518

Training MSE - 1300478.9218781218
Testing MSE - 1198182.4105224987

Training RMSE - 1140.3854268965918
Testing RMSE - 1094.6151883298983



According to the R2, the model can predict about 56% of the variance of the target.

Given that the mean sale is around $2200, an RMSE of 1140 is quite large.

##Descision Tree Regressor

In [ ]:
#create decision tree regressor
dec_tree = DecisionTreeRegressor(random_state = 42)

In [ ]:
#fit data
dec_tree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [ ]:
#make predictions
train_p = dec_tree.predict(X_train)
test_p = dec_tree.predict(X_test)

In [ ]:
#get scores
train_score = dec_tree.score(X_train, y_train)
test_score = dec_tree.score(X_test, y_test)
print(train_score)
print(test_score)

1.0
0.17210839420595614


The training score is perfect, while the test score is very low. This implies overfitting.

##Hypertuning

In [ ]:
#get parameters
dec_tree.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 42,
 'splitter': 'best'}

In [ ]:
#get depth
dec_tree.get_depth()

42

The max depth was 42, so we need the try lower depths to find the best model.

In [ ]:
#iterate through all depths and find the highest test score
depths = list(range(2, 42))
scores = pd.DataFrame(index=depths, columns=['Test Score','Train Score'])
for depth in depths:
  dec_tree = DecisionTreeRegressor(max_depth=depth, random_state=42)
  dec_tree.fit(X_train, y_train)
  train_score = dec_tree.score(X_train, y_train)
  test_score = dec_tree.score(X_test, y_test)
  scores.loc[depth, 'Train Score'] = train_score
  scores.loc[depth, 'Test Score'] = test_score
sorted_scores = scores.sort_values(by='Test Score', ascending=False)
print(sorted_scores.head())

  Test Score Train Score
5    0.59471     0.60394
4   0.584005    0.582625
6   0.582356    0.615072
7   0.578891    0.626453
8   0.563151    0.642791


In [ ]:
#create best decision tree regressor
best_dec_tree = DecisionTreeRegressor(random_state = 42, max_depth=5)

In [ ]:
#fit data
best_dec_tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5, random_state=42)

In [ ]:
#make predictions
train_p = best_dec_tree.predict(X_train)
test_p = best_dec_tree.predict(X_test)

In [ ]:
#get scores
train_score = best_dec_tree.score(X_train, y_train)
test_score = best_dec_tree.score(X_test, y_test)
print(train_score)
print(test_score)

0.6039397477322956
0.5947099753159972


In [ ]:
#apply the function to get metrics
get_metrics(best_dec_tree)

Training R2 - 0.6039397477322956
Testing R2 - 0.5947099753159972

Training MAE - 762.6101695559576
Testing MAE - 738.3173097797822

Training MSE - 1172122.7729098853
Testing MSE - 1118185.973077762

Training RMSE - 1082.6461900869947
Testing RMSE - 1057.4431299496734



I would use the decision tree over the linear regression model. The decision tree is a better model, but only slightly. It's R2 is higher meaning that it can predict more variance in the target. Also, it's RSME is lower, which means it has lower errors.